In [1]:
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
#did not perform mallet

In [2]:
# Run in python console
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dastous\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [4]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [5]:
df_review=pd.read_excel("../CannaConnect/Dataset/description_clean.xlsx")
df=df_review.Description.astype(str)
df.head()

0     og   hybrid strain pack strong punch name sup...
1     aloha white widow especially potent cut white...
2     sativa dominant hybrid bred spain medical see...
3     dawgs hybrid g chemdawg genetics bred canadia...
4    known kosher tangie k gold  indica dominant hy...
Name: Description, dtype: object

In [6]:
# Convert to list
data = df.values.tolist()

In [7]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['og', 'hybrid', 'strain', 'pack', 'strong', 'punch', 'name', 'supposedly', 'refers', 'strength', 'high', 'price', 'first', 'started', 'showing', 'hollywood', 'plant', 'og', 'tends', 'produce', 'large', 'dark', 'green', 'bud', 'stem', 'user', 'report', 'strong', 'body', 'effect', 'indica', 'pain', 'relief', 'alert', 'cerebral', 'feeling', 'thanks', 'sativa', 'side']]


In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['og', 'hybrid', 'strain', 'pack', 'strong', 'punch', 'name', 'supposedly', 'refers', 'strength', 'high', 'price', 'first', 'started', 'showing', 'hollywood', 'plant', 'og', 'tends', 'produce', 'large', 'dark', 'green', 'bud', 'stem', 'user_report', 'strong', 'body', 'effect', 'indica', 'pain', 'relief', 'alert', 'cerebral', 'feeling', 'thanks', 'sativa', 'side']


In [9]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [10]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['hybrid', 'strain', 'pack', 'strong', 'punch', 'name', 'supposedly', 'refer', 'strength', 'high', 'price', 'first', 'start', 'show', 'hollywood', 'plant', 'tend', 'produce', 'large', 'dark', 'green', 'bud', 'stem', 'user_report', 'strong', 'body', 'effect', 'indica', 'pain', 'relief', 'alert', 'cerebral', 'feeling', 'thank', 'side']]


In [11]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 2), (30, 1), (31, 1), (32, 1), (33, 1)]]


In [12]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('alert', 1),
  ('body', 1),
  ('bud', 1),
  ('cerebral', 1),
  ('dark', 1),
  ('effect', 1),
  ('feeling', 1),
  ('first', 1),
  ('green', 1),
  ('high', 1),
  ('hollywood', 1),
  ('hybrid', 1),
  ('indica', 1),
  ('large', 1),
  ('name', 1),
  ('pack', 1),
  ('pain', 1),
  ('plant', 1),
  ('price', 1),
  ('produce', 1),
  ('punch', 1),
  ('refer', 1),
  ('relief', 1),
  ('show', 1),
  ('side', 1),
  ('start', 1),
  ('stem', 1),
  ('strain', 1),
  ('strength', 1),
  ('strong', 2),
  ('supposedly', 1),
  ('tend', 1),
  ('thank', 1),
  ('user_report', 1)]]

In [13]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [14]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.034*"strain" + 0.027*"high" + 0.027*"bud" + 0.023*"flower" + 0.019*"seed" '
  '+ 0.019*"haze" + 0.015*"plant" + 0.015*"time" + 0.015*"cannabis" + '
  '0.015*"breed"'),
 (1,
  '0.031*"cbd" + 0.025*"strawberry" + 0.023*"super" + 0.017*"result" + '
  '0.016*"thc" + 0.013*"dream" + 0.013*"tangerine" + 0.012*"place" + '
  '0.012*"tangie" + 0.012*"relieve"'),
 (2,
  '0.125*"white" + 0.027*"trichome" + 0.026*"white_widow" + 0.017*"coat" + '
  '0.016*"trainwreck" + 0.016*"fire" + 0.014*"crystal" + 0.014*"alien" + '
  '0.013*"stimulation" + 0.012*"similar"'),
 (3,
  '0.098*"cooky" + 0.043*"girl_scout" + 0.026*"wookie" + 0.025*"limb" + '
  '0.024*"imbue" + 0.020*"gsc" + 0.018*"speak" + 0.017*"cherry_pie" + '
  '0.016*"cookie" + 0.013*"aka"'),
 (4,
  '0.067*"short" + 0.041*"outdoor" + 0.023*"october" + 0.018*"exceptional" + '
  '0.016*"truth" + 0.016*"gene" + 0.016*"lsd" + 0.015*"breeding" + '
  '0.013*"project" + 0.013*"latent"'),
 (5,
  '0.035*"slipper" + 0.034*"flowering_period" + 0.

In [15]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.759097059193203

Coherence Score:  0.401098027535922


In [16]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\dastous\Anaconda3\envs\py3.6\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.353961 -0.037198       1        1  44.725246
0      0.324593 -0.040842       2        1  26.917398
1     -0.046773  0.290934       3        1   7.112715
6      0.008495  0.062100       4        1   6.865609
7     -0.032727  0.128489       5        1   6.417228
2     -0.096111 -0.252038       6        1   5.567465
3     -0.136597 -0.045952       7        1   1.159778
4     -0.129982 -0.038426       8        1   0.664286
5     -0.124083 -0.034408       9        1   0.344298
9     -0.120776 -0.032659      10        1   0.225967, topic_info=     Category         Freq           Term        Total  loglift  logprob
570   Default   684.000000          white   684.000000  30.0000  30.0000
207   Default   598.000000           sour   598.000000  29.0000  29.0000
41    Default   433.000000         diesel   433.000000  28.0000  28.0000
5     Default  1676.000000         effect  1676.000000  27.0000  27.0000
27    Default  3095.000000         strain  3095.000000  26.0000  26.0000
2     Default   723.000000            bud   723.000000  25.0000  25.0000
9     Default   722.000000           high   722.000000  24.0000  24.0000
145   Default  1076.000000           kush  1076.000000  23.0000  23.0000
73    Default   703.000000           seed   703.000000  22.0000  22.0000
139   Default   602.000000         flower   602.000000  21.0000  21.0000
700   Default   500.000000           haze   500.000000  20.0000  20.0000
237   Default   250.000000            cbd   250.000000  19.0000  19.0000
85    Default  1088.000000       dominant  1088.000000  18.0000  18.0000
12    Default  1434.000000         indica  1434.000000  17.0000  17.0000
1     Default   694.000000           body   694.000000  16.0000  16.0000
119   Default   384.000000         citrus   384.000000  15.0000  15.0000
17    Default   400.000000          plant   400.000000  14.0000  14.0000
1033  Default   112.000000          cooky   112.000000  13.0000  13.0000
370   Default   393.000000           time   393.000000  12.0000  12.0000
74    Default   360.000000          skunk   360.000000  11.0000  11.0000
110   Default   257.000000            thc   257.000000  10.0000  10.0000
1576  Default   173.000000     strawberry   173.000000   9.0000   9.0000
76    Default   157.000000       trichome   157.000000   8.0000   8.0000
50    Default   351.000000           grow   351.000000   7.0000   7.0000
132   Default  1048.000000          aroma  1048.000000   6.0000   6.0000
1843  Default   159.000000          super   159.000000   5.0000   5.0000
152   Default   156.000000         orange   156.000000   4.0000   4.0000
19    Default   328.000000        produce   328.000000   3.0000   3.0000
236   Default   500.000000       cannabis   500.000000   2.0000   2.0000
78    Default   141.000000    white_widow   141.000000   1.0000   1.0000
...       ...          ...            ...          ...      ...      ...
1357  Topic10     2.725955           sell     3.468047   5.8518  -4.3998
4841  Topic10     1.917757        inbreed     2.659322   5.7656  -4.7514
744   Topic10     1.635079   united_state     2.376590   5.7186  -4.9109
2114  Topic10     1.581292       possibly     2.322949   5.7079  -4.9443
514   Topic10     1.210829          shiva     1.952238   5.6149  -5.2113
3402  Topic10     0.591072  international     1.332505   5.2797  -5.9284
1677  Topic10     0.153640         stocky     0.895021   4.3303  -7.2757
510   Topic10     0.145875           jock     0.887251   4.2872  -7.3276
508   Topic10     0.145875         horror     0.887251   4.2872  -7.3276
1697  Topic10     0.092817    expectation     0.834385   3.8965  -7.7797
5203  Topic10     0.036581         pugent     0.828488   2.9725  -8.7108
5204  Topic10     0.036581          queso     0.828488   2.9725  -8.7108
5199  Topic10     0.036581       europian     0.828488   2.9725  -8.7108
5198  Topic10     0.0365